<a href="https://colab.research.google.com/github/bobby-py2002/FoodVision/blob/main/scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

scrape.py



In [8]:
import requests
from bs4 import BeautifulSoup
import json

def clean_text(text):

    return ' '.join(text.split()).replace('\n', ' ').strip()

def scrape_recipe(url):

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    return {
        'name': soup.find('h1').text.strip() if soup.find('h1') else None,
        'ingredients': [clean_text(ing.text) for ing in soup.select('.ingredient-list li')],
        'instructions': [clean_text(ins.text) for ins in soup.select('.direction-list li')]
    }


recipes = []
urls = ['https://www.food.com/recipe/bagel-french-toast-casserole-362199']

for url in urls:
    try:
        recipes.append(scrape_recipe(url))
    except Exception as e:
        print(f"Error: {e}")

with open('/home/b_o_bby/FoodVision/data/processed/recipes.json', 'w', encoding='utf-8') as f:
    json.dump(recipes, f, indent=2, ensure_ascii=False)